In [1]:
%matplotlib inline
from os import listdir, path
from requests import get
import urllib3
from tqdm import tqdm
from lxml import etree
import pandas as pd
import warnings
import itertools
warnings.filterwarnings('ignore')
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
for year in tqdm(range(1994, 2018)):
    stop = False
    i=0
    while stop == False:
        i+=1
        for j in range(1, 200):
            file = "h-tk-%s-%s-%s" %(str(year)+str(year+1), str(i), str(j))
            if not path.isfile('HandelingenTK/%s.xml' %file):
                url = "https://zoek.officielebekendmakingen.nl/%s" %file
                resp = get(url + ".xml", verify=False, allow_redirects=False)
                if resp.status_code == 200:
                    hand_file = open('HandelingenTK/%s.xml' %file, 'wb')
                    hand_file.write(resp.content)
                    hand_file.close()
                    resp = get(url+"/metadata.xml", verify=False, \
                                                        allow_redirects=False)
                    meta_file = open('HandelingenTKmeta/%smetadata.xml' %file, 'wb')
                    meta_file.write(resp.content)
                    meta_file.close()    
                elif j==1:
                    stop = True
                    break
                else:
                    break

100%|█████████████████████████████████████████████████████████████████████████████████| 24/24 [54:22<00:00, 135.96s/it]


In [3]:
enddict = {}
i = 0
for file in tqdm(listdir('HandelingenTK/')):
    context = etree.iterparse('HandelingenTK/' + file,  tag='spreekbeurt')
    for _, elem in context:
        
        surname = elem.findtext('spreker/naam/achternaam')
        party = elem.findtext('spreker/politiek')
        text = ' '.join([elem2.findtext('al') for elem2 \
                         in elem.findall('tekst/al-groep') \
                          if elem2.findtext('al') is not None])
        enddict[i] = [surname, party, text, file]
        del surname, party, text
        i += 1
df = pd.DataFrame.from_dict(enddict, orient='index')
df = df.rename(columns={0:'achternaam', 1:'partij', 2:'tekst', 3:'file'})

100%|███████████████████████████████████████████████████████████████████████████| 14230/14230 [00:33<00:00, 418.72it/s]


In [4]:
enddict = {}
for file in tqdm(listdir('HandelingenTKmeta')):
    tags = []
    datum = None
    context = etree.iterparse('HandelingenTKmeta/' + file,  tag='metadata')
    for _, elem in context:
        if elem.get('name') == "OVERHEIDop.datumVergadering":
            datum = elem.get('content')
        elif elem.get('name') == "OVERHEID.category":
            tags.append(elem.get('content'))
        enddict[file[:-12]+'.xml'] = [datum, tags]
df2 = pd.DataFrame.from_dict(enddict, orient='index')
df2 = df2.rename(columns={0:'datum', 1:'tags'})

100%|██████████████████████████████████████████████████████████████████████████| 14227/14227 [00:08<00:00, 1637.62it/s]


In [5]:
combineddf = df.join(df2, on='file')
combineddf.to_csv("Handelingen.csv")
combineddf.head(5)

,achternaam,partij,tekst,file,datum,tags
0,voorzitter,None,,h-tk-20102011-100-1.xml,2011-06-30,[Bestuur | Parlement]
1,voorzitter,None,,h-tk-20102011-100-10.xml,2011-06-30,[Recht | Strafrecht]
2,voorzitter,None,,h-tk-20102011-100-10.xml,2011-06-30,[Recht | Strafrecht]
3,voorzitter,None,,h-tk-20102011-100-10.xml,2011-06-30,[Recht | Strafrecht]
4,voorzitter,None,,h-tk-20102011-100-11.xml,2011-06-30,[Recht | Strafrecht]
